# Collaborative Filtering

In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

## Data Info

In [2]:
df = pd.read_csv('../data/00_raw/transactions_train.csv', nrows=100)

In [3]:
articles = pd.read_csv('../data/00_raw/articles.csv')

In [4]:
articles = articles[['article_id', 'colour_group_code', 'colour_group_name']]

In [5]:
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [6]:
data = df.groupby(['customer_id', 'article_id'])['customer_id'].count().reset_index(name='total_bought')

In [7]:
data = data.merge(articles, how='left', on='article_id')

In [8]:
# Encode the genres data
encoder = LabelEncoder()
encoder.fit(data['customer_id'])
data['encoded_customer_id'] = encoder.transform(data['customer_id'])

In [9]:
X = data.loc[:,['encoded_customer_id','article_id','colour_group_code']]
y = data.loc[:,'total_bought']

# Split our data into training and test sets
X_train, X_val, y_train, y_val = train_test_split(X,y,random_state=0, test_size=0.2)

In [10]:
def prep_dataloaders(X_train,y_train,X_val,y_val,batch_size):
    # Convert training and test data to TensorDatasets
    trainset = TensorDataset(torch.from_numpy(np.array(X_train)).long(), 
                            torch.from_numpy(np.array(y_train)).float())
    valset = TensorDataset(torch.from_numpy(np.array(X_val)).long(), 
                            torch.from_numpy(np.array(y_val)).float())

    # Create Dataloaders for our training and test data to allow us to iterate over minibatches 
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False)

    return trainloader, valloader

batchsize = 64
trainloader,valloader = prep_dataloaders(X_train,y_train,X_val,y_val,batchsize)

In [11]:
class NNHybridFiltering(nn.Module):
    
    def __init__(self, n_users, n_items, n_genres, embdim_users, embdim_items, embdim_genres, n_activations, rating_range):
        super().__init__()
        self.user_embeddings = nn.Embedding(num_embeddings=n_users,embedding_dim=embdim_users)
        self.item_embeddings = nn.Embedding(num_embeddings=n_items,embedding_dim=embdim_items)
        self.genre_embeddings = nn.Embedding(num_embeddings=n_genres,embedding_dim=embdim_genres)
        self.fc1 = nn.Linear(embdim_users+embdim_items+embdim_genres,n_activations)
        self.fc2 = nn.Linear(n_activations,1)
        self.rating_range = rating_range

    def forward(self, X):
        # Get embeddings for minibatch
        embedded_users = self.user_embeddings(X[:,0])
        embedded_items = self.item_embeddings(X[:,1])
        embedded_genres = self.genre_embeddings(X[:,2])
        # Concatenate user, item and genre embeddings
        embeddings = torch.cat([embedded_users,embedded_items,embedded_genres],dim=1)
        # Pass embeddings through network
        preds = self.fc1(embeddings)
        preds = F.relu(preds)
        preds = self.fc2(preds)
        # Scale predicted ratings to target-range [low,high]
        preds = torch.sigmoid(preds) * (self.rating_range[1]-self.rating_range[0]) + self.rating_range[0]
        return preds


In [12]:
def train_model(model, criterion, optimizer, dataloaders, device, num_epochs=5, scheduler=None):
    model = model.to(device) # Send model to GPU if available
    since = time.time()

    costpaths = {'train':[],'val':[]}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            # Get the inputs and labels, and send to GPU if available
            for (inputs,labels) in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the weight gradients
                optimizer.zero_grad()

                # Forward pass to get outputs and calculate loss
                # Track gradient only for training data
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model.forward(inputs).view(-1)
                    loss = criterion(outputs, labels)

                    # Backpropagation to get the gradients with respect to each weight
                    # Only if in train
                    if phase == 'train':
                        loss.backward()
                        # Update the weights
                        optimizer.step()

                # Convert loss into a scalar and add it to running_loss
                running_loss += np.sqrt(loss.item()) * labels.size(0)

            # Step along learning rate scheduler when in train
            if (phase == 'train') and (scheduler is not None):
                scheduler.step()

            # Calculate and display average loss and accuracy for the epoch
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            costpaths[phase].append(epoch_loss)
            print('{} loss: {:.4f}'.format(phase, epoch_loss))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    return costpaths


In [ ]:
# Train the model
dataloaders = {'train':trainloader, 'val':valloader}
n_users = X.loc[:,'encoded_customer_id'].max()+1
n_items = X.loc[:,'article_id'].max()+1
n_genres = X.loc[:,'colour_group_code'].max()+1
model = NNHybridFiltering(n_users,
                       n_items,
                       n_genres,
                       embdim_users=50, 
                       embdim_items=50, 
                       embdim_genres=25,
                       n_activations = 100,
                       rating_range=[0.,5.])
criterion = nn.MSELoss()
lr=0.001
n_epochs=10
wd=1e-3
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cost_paths = train_model(model,criterion,optimizer,dataloaders, device,n_epochs, scheduler=None)


## Collaborative Filtering

### Creating collab filter df

In [ ]:
ohe_article = pd.get_dummies(df['product_code'])

In [ ]:
len(df.customer_id.unique())

In [ ]:
article_names = list(ohe_article.columns)

In [ ]:
cf_df = pd.concat([df['customer_id'], ohe_article], axis=1)

In [ ]:
cf_df = cf_df.groupby(['customer_id'])[article_names].sum().reset_index()

In [ ]:
len(cf_df.columns)

In [ ]:
cf_df.head()

### Test train split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = cf_df['customer_id']
X = cf_df.drop(columns='customer_id')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
rankings = cosine_similarity(X_test, X_train, dense_output=False)

In [ ]:
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [ ]:
recommendations = {}
for i, pred in enumerate(rankings):
    idxs = list(np.argpartition(np.array(pred), -20)[-20:])
    recs = []
    for idx in idxs:
        cid = y_train.loc[[idx]].values[0]
        rec = list(df[df['customer_id'] == cid].groupby(['article_id'])['customer_id'].count().sort_values(ascending=False).index.values.astype('int'))
        recs.extend(rec)

    recs = list(set(recs))
    
    try:
        recs = recs[:12]
    except:
        pass
    
    cur_customer = y_test.iloc[[i]].values[0]
    recommendations[cur_customer] = recs

In [ ]:
mAP_list = []
for customer in recommendations:
    top_recs = recommendations[customer]
    actual_bought = list(df[df['customer_id'] == customer].groupby(['article_id'])['customer_id'].count().sort_values(ascending=False).index.values.astype('int'))
    correct = len(set(actual_bought) - set(top_recs))
    ap = correct / len(actual_bought)
    mAP_list.append(ap)

In [ ]:
np.array(mAP_list).mean()

In [ ]:
pop_mAP_list = []
for customer in recommendations:
    pop_recs = df.groupby(['article_id'])['customer_id'].count().sort_values(ascending=False).index.values[:12]
    actual_bought = list(df[df['customer_id'] == customer].groupby(['article_id'])['customer_id'].count().sort_values(ascending=False).index.values.astype('int'))
    correct = len(set(actual_bought) - set(pop_recs))
    ap = correct / len(actual_bought)
    pop_mAP_list.append(ap)

In [ ]:
np.array(pop_mAP_list).mean()